In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

In [ ]:
%%bash
mkdir -p data write
cd data
test -f pbmc3k_filtered_gene_bc_matrices.tar.gz || curl https://cf.10xgenomics.com/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -o pbmc3k_filtered_gene_bc_matrices.tar.gz
tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz

In [ ]:
sc.settings.verbosity = 3
sc.set_figure_params(dpi=80, facecolor="white")
sc.logging.print_header()

In [ ]:
results_file = "write/pbmc3k.h5ad"

In [ ]:
adata = sc.read_10x_mtx(
    "data/filtered_gene_bc_matrices/hg19/", 
    var_names="gene_symbols", 
    cache=True, 
)

In [ ]:
adata

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata

## Data Preprocessing

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata = adata[
    (adata.obs.n_genes_by_counts < 2500) & (adata.obs.n_genes_by_counts > 200) & (adata.obs.pct_counts_mt < 5),
    :,
].copy()
adata.layers["counts"] = adata.X.copy()

In [ ]:
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    layer="counts",
    n_top_genes=2000,
    min_mean=0.0125,
    max_mean=3,
    min_disp=0.5,
    flavor="seurat_v3",
)

In [ ]:
adata.layers["scaled"] = adata.X.toarray()
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"], layer="scaled")
sc.pp.scale(adata, max_value=10, layer="scaled")

## Principal Component Analysis (PCA)

In [ ]:
sc.pp.pca(adata, layer="scaled", svd_solver="arpack")

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=20)

## Computing the neighborhood graph

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## UMAP

In [ ]:
sc.tl.umap(adata)

In [ ]:
# Using normalized gene expression, 
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"])

In [ ]:
# Using raw gene expression, 
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"], layer="counts")

In [ ]:
# Using scaled gene expression, 
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"], layer="scaled")

## Graph-based Clustering

In [ ]:
sc.tl.leiden(
    adata,
    resolution=0.7,
    random_state=0,
    flavor="igraph",
    n_iterations=2,
    directed=False,
)
adata.obs["leiden"] = adata.obs["leiden"].copy()
adata.uns["leiden"] = adata.uns["leiden"].copy()
adata.obsm["X_umap"] = adata.obsm["X_umap"].copy()

In [ ]:
sc.pl.umap(adata, color=["leiden", "CD14", "NKG7"])

## Finding marker genes

In [ ]:
sc.tl.rank_genes_groups(adata, "leiden", mask_var="highly_variable", method="t-test")
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
# Savepoint
adata.write(results_file)

In [ ]:
marker_genes = [
    *["IL7R", "CD79A", "MS4A1", "CD8A", "CD8B", "LYZ", "CD14"],
    *["LGALS3", "S100A8", "GNLY", "NKG7", "KLRB1"],
    *["FCGR3A", "MS4A7", "FCER1A", "CST3", "PPBP"],
]

In [ ]:
pd.DataFrame(adata.uns["rank_genes_groups"]["names"]).head(5)

In [ ]:
result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
pd.DataFrame({f"{group}_{key[:1]}": result[key][group] for group in groups for key in ["names", "pvals"]}).head(5)

In [ ]:
sc.pl.violin(adata, ["CST3", "NKG7", "PPBP"], groupby="leiden")

## Cell type annotation

In [ ]:
new_cluster_names = [
    "CD4 T",
    "B",
    "CD14+ Monocytes",
    "NK",
    "CD8 T",
    "FCGR3A+ Monocytes",
    "Dendritic",
    "Megakaryocytes",
]
adata.rename_categories("leiden", new_cluster_names)

In [ ]:
sc.pl.umap(adata, color="leiden", legend_loc="on data", title="", frameon=False, save=".pdf")

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby="leiden")

In [ ]:
adata

## Differentially expressed genes

In [ ]:
sc.tl.rank_genes_groups(
    adata,
    "leiden",
    mask_var="highly_variable",
    groups=["CD4 T"],
    reference="CD8 T",
    method="wilcoxon",
)
sc.pl.rank_genes_groups(adata, groups=["CD4 T"], n_genes=20)

In [ ]:
result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
pd.DataFrame({f"{group}_{key[:1]}": result[key][group] for group in groups for key in ["names", "pvals"]}).head(5)